# Grad CAM

In [1]:
import os
import sys
import json
import time
import pickle
import argparse
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

sys.path.append("../")
from utils import *
from GradCAM.modeltrain import train_model, select_model, eval_model
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchcam.utils import overlay_mask

from PIL import Image
# from torchcam.methods import GradCAMpp, GradCAM
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image

import warnings
warnings.filterwarnings("ignore")

In [36]:
dataset_name = "cifar100"
datasetpath = f"../datasets/{dataset_name}"
saliencymappath = f"../datasets/{dataset_name}_saliency_maps"
n_classes = int(dataset_name[5:])
model_name = "vgg16"
savepath = f"../GradCAM/saved_models/{dataset_name}"
criterion = nn.CrossEntropyLoss() 

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## MODEL

In [4]:
reference_models = 4
batch_size = 256
learning_rate = 0.001
epochs = 32

In [5]:
# Load CIFAR-10 dataset
if dataset_name=="cifar10":
    train_dataset = datasets.CIFAR10(root=datasetpath, train=True, transform=transform, download=True)
    test_dataset = datasets.CIFAR10(root=datasetpath, train=False, transform=transform, download=True)
elif dataset_name=="cifar100":
    train_dataset = datasets.CIFAR100(root=datasetpath, train=True, transform=transform, download=True)
    test_dataset = datasets.CIFAR100(root=datasetpath, train=False, transform=transform, download=True)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
model = select_model(model_name, n_classes)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.6, weight_decay=5e-4)
if not os.path.exists(savepath):
    os.makedirs(savepath)
modelpath = os.path.join(savepath,model_name+".pth")
train_model(model, train_loader, test_loader, optimizer, device, modelpath, epochs)

Epoch 1 -> Training Accuracy: 0.1060, Validation Accuracy: 0.2899
New best model saved with accuracy: 0.2899
Epoch [1/32], Loss: 4.0375


Epoch 2 -> Training Accuracy: 0.3851, Validation Accuracy: 0.4295
New best model saved with accuracy: 0.4295
Epoch [2/32], Loss: 2.3186


Epoch 3 -> Training Accuracy: 0.4783, Validation Accuracy: 0.4735
New best model saved with accuracy: 0.4735
Epoch [3/32], Loss: 1.8756


Epoch 4 -> Training Accuracy: 0.5292, Validation Accuracy: 0.5037
New best model saved with accuracy: 0.5037
Epoch [4/32], Loss: 1.6580


Epoch 5 -> Training Accuracy: 0.5685, Validation Accuracy: 0.5231
New best model saved with accuracy: 0.5231
Epoch [5/32], Loss: 1.5046


...

Epoch 28 -> Training Accuracy: 0.9913, Validation Accuracy: 0.5725
Epoch [28/32], Loss: 0.0474


Epoch 29 -> Training Accuracy: 0.9944, Validation Accuracy: 0.5689
Epoch [29/32], Loss: 0.0344


Epoch 30 -> Training Accuracy: 0.9947, Validation Accuracy: 0.5672
Epoch [30/32], Loss: 0.0302


Epoch 31 -> Training Accuracy: 0.9965, Validation Accuracy: 0.5758
New best model saved with accuracy: 0.5758
Epoch [31/32], Loss: 0.0238


Epoch 32 -> Training Accuracy: 0.9975, Validation Accuracy: 0.5777
New best model saved with accuracy: 0.5777
Epoch [32/32], Loss: 0.0180


## GRAD-CAM

In [38]:
if dataset_name=="cifar10":
    test_dataset = datasets.CIFAR10(root=datasetpath, train=False, download=True, transform=transform)
elif dataset_name=="cifar100":
    test_dataset = datasets.CIFAR100(root=datasetpath, train=False, download=True, transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified


In [39]:
model = select_model(model_name, n_classes)
model.load_state_dict(torch.load(os.path.join(savepath, model_name+".pth")))
model = model.to(device)
model.eval()
eval_model(model, test_loader, criterion, device)

(0.5777, 0.056762674176692965)

In [40]:
if dataset_name=="cifar10":
    train = datasets.CIFAR10(root=datasetpath, train=True, download=True, transform=transform)
    test = datasets.CIFAR10(root=datasetpath, train=False, download=True, transform=transform)
elif dataset_name=="cifar100":
    train = datasets.CIFAR100(root=datasetpath, train=True, download=True, transform=transform)
    test = datasets.CIFAR100(root=datasetpath, train=False, download=True, transform=transform)
full = torch.utils.data.ConcatDataset([train, test])
data_loader = DataLoader(full, batch_size=1, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [41]:
os.makedirs(saliencymappath, exist_ok=True)
for class_idx in range(n_classes):
    os.makedirs(os.path.join(saliencymappath, str(class_idx)), exist_ok=True)

In [42]:
# JSON Metadata File
metadata = []
cam = GradCAM(model=model, target_layers=[model.features[-1]])

# Step 4: Generate and Save Saliency Maps
for i, (images, labels) in enumerate(data_loader):
    images = images.to(device)
    label = labels.to(device)
    # Get Saliency Map
    targets = [ClassifierOutputTarget(label)]
    grayscale_cam = cam(input_tensor=images, targets=targets)
    grayscale_cam = grayscale_cam[0, :]
    
    file_path = os.path.join(saliencymappath, str(label.item()), f"saliency_map_{i}.npy")
    os.makedirs(os.path.dirname(file_path), exist_ok=True)  # Ensure the directory exists
    np.save(file_path, grayscale_cam)  # Save the saliency map as a .npy file
    
    # Append metadata (index, label, and file path)
    metadata.append({"index": i, "label": str(label.item()), "file_path": file_path})
    
    if i % 1000 == 0:
        print(f"Processed {i}/{len(full)} images.")

# Save Metadata as JSON
with open(os.path.join(saliencymappath, "metadata.json"), "w") as f:
    json.dump(metadata, f, indent=4)

print("Saliency maps and metadata saved successfully!")


Processed 0/60000 images.
Processed 1000/60000 images.
Processed 2000/60000 images.
Processed 3000/60000 images.
Processed 4000/60000 images.
Processed 5000/60000 images.
Processed 6000/60000 images.
Processed 7000/60000 images.
Processed 8000/60000 images.
Processed 9000/60000 images.
Processed 10000/60000 images.
Processed 11000/60000 images.
Processed 12000/60000 images.
Processed 13000/60000 images.
Processed 14000/60000 images.
Processed 15000/60000 images.
Processed 16000/60000 images.
Processed 17000/60000 images.
Processed 18000/60000 images.
Processed 19000/60000 images.
Processed 20000/60000 images.
Processed 21000/60000 images.
Processed 22000/60000 images.
Processed 23000/60000 images.
Processed 24000/60000 images.
Processed 25000/60000 images.
Processed 26000/60000 images.
Processed 27000/60000 images.
Processed 28000/60000 images.
Processed 29000/60000 images.
Processed 30000/60000 images.
Processed 31000/60000 images.
Processed 32000/60000 images.
Processed 33000/60000 i

In [ ]:
# # Overlay CAM on Image
    # original_image = images[0].cpu().permute(1, 2, 0).numpy()
    # original_image = (original_image - original_image.min()) / (original_image.max() - original_image.min())
    # cam_image = show_cam_on_image(original_image, grayscale_cam, use_rgb=True)
    
    # Save Saliency Map
    # file_path = os.path.join(saliencymappath, str(label.item()), f"saliency_map_{i}.png")
    # # cam_image1 = Image.fromarray(cam_image)
    # # cam_image.save(file_path)
    # grayscale_cam1 = Image.fromarray(grayscale_cam[:,:,np.newaxis])
    # grayscale_cam1.save(file_path)